# CHSH game

### Alice & Bob vs. Charlie
1. Charile sends to Alice $x\in\{0,1\}$, and also sends to Bob $y\in\{0,1 \}$.
2. After Alice has received $x$, she returns $a$.
3. After Bob has received $y$, he returns $b$.
4. If $x\land y = a\otimes b$ is satisfied,  Alice and Bob win, otherwise they loses

### Win rate $S$
- classical case : $S\le0.75$
- quantum case : $S\gt0.75$ is possible

In [1]:
import random as rand
import qiskit
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, quantum_info

## Classical strategy

Let's first play the game using a deterministic classical strategy. Remember, this means that Alice can only devise a strategy based on her value of the input bit x and Bob according to its input value y and they cannot communicate with each other. Here are a few possible strategies:

1) output = input

2) output = NOT(input)

3) always output = 1

4) always output = 0


In [2]:
def cplayer_output(strategy, inp):
    if(strategy == 1):
        return inp
    
    elif(strategy == 2):
        return abs(inp-1)
    
    elif(strategy == 3):
        return 1
        
    elif(strategy == 4):
        return 0
            
    else:
        print("INVALID choice")
        return 100

In [3]:
# Pick Alice's classical strategy
A_st = int(input('select the classical strategy for Alice, input 1,2,3 or 4 to pick one of the strategies listed above '))

# Pick Bob's classical strategy
B_st = int(input('select the classical strategy for Bob, input 1,2,3 or 4 to pick one of the strategies listed above '))

select the classical strategy for Alice, input 1,2,3 or 4 to pick one of the strategies listed above 4
select the classical strategy for Bob, input 1,2,3 or 4 to pick one of the strategies listed above 4


In [4]:
# fixes the numbers of games to be played
N=1000

# initializes counters used to keep track of the numbers of games won and played by Alice an Bob
cont_win = 0 # counts games won
cont_tot = 0 # counts games played


# play the game N times
for i in range(N):
    
    # generates two random input from the refree, x and y, to be given to Alice and Bob
    random_num1 = rand.random() # first random number
    random_num2 = rand.random() # second random number

    if(random_num1 >= 1/2): # converts the first random number to 0 or 1
        x = 0
    else: x = 1

    if(random_num2 >= 1/2): # converts the second random number to 0 or 1
        y = 0
    else: y = 1
    
    
    # generates Alice's and Bob's output
    a = cplayer_output(A_st, x) # Alice's output
    
    b = cplayer_output(B_st, y) # Bob's output


    # check if the condition for winning the game is met
    if(x*y == a^b):
        cont_win += 1 # increase thes won games' counter if the condition to win the game is met
    
    cont_tot += 1 # increases the played games' counter

Prob_win = cont_win/cont_tot # winning probability

print('Alice and Bob won the game with probability: ', Prob_win*100, '%')

Alice and Bob won the game with probability:  75.6 %


## Quantum strategy

As a quantum strategy, Alice and Bob play the game by sharing an entangled pair of qubit. That is, two qubits are brought together to form an entangled pair and then Alice and Bob are given one each. Beware, even though the entangled qubits influence each other, it is not possible for Alice and Bob to use them to communicate with each other during the game.

According to the input they receive from the refree they will rotate their entangled qubit and then measure it. For example, pick one of the following strategies:

1) don't rotate the qubit

2) rotate a random amount

3) rotate to maximize winning probability

In [5]:
def qAlice_output(strategy, inp):
    if(strategy == 1):
        return 0
    
    elif(strategy == 2):
        return rand.uniform(0,2*np.pi)
    
    elif(strategy == 3):
        if(inp == 0):
            return 0
        elif(inp == 1):
            return np.pi/2        
            
    else:
        print("INVALID choice")
        return 100
    

def qBob_output(strategy, inp):
    if(strategy == 1):
        return 0
    
    elif(strategy == 2):
        return rand.uniform(0,2*np.pi)
    
    elif(strategy == 3):
        if(inp == 0):
            return np.pi/4
        elif(inp == 1):
            return -np.pi/4        
            
    else:
        print("INVALID choice")
        return 100

In [6]:
# Alice's strategy
qA_st = int(input('select the quantum strategy for Alice, input 1,2 or 3 to pick one of the strategies listed above: '))

# Bob's strategy
qB_st = int(input('select the quantum strategy for Bob, input 1,2 or 3 to pick one of the strategies listed above: '))

select the quantum strategy for Alice, input 1,2 or 3 to pick one of the strategies listed above: 3
select the quantum strategy for Bob, input 1,2 or 3 to pick one of the strategies listed above: 3


In [7]:
# set parameters of the quantum run of the game 
shots = 1 # set how many times the circuit is run, accumulating statistics about the measurement outcomes 
backend = Aer.get_backend('qasm_simulator') # set the machine where the quantum circuit is to be run   

#fixes the numbers of games to be played
N=1000

# initializes counters used to keep track of the numbers of games won and played by Alice an Bob
cont_win = 0 # counts games won
cont_tot = 0 # counts games played

#play N games
for i in range(N):

    # creates quantum circuit, to write a quantum algorithm we will add gates to the circuit
    game = QuantumCircuit(2)   
    
    # These gates prepare the entangled Bell pair to be shared by Alice and Bob as part of their quantum strategy
    # Alice will have qubit 0 and Bob will have qubit 1
    game.h(0)
    game.cx(0,1)

    # generates two random input from the refree, x and y, to be given to Alice and Bob
    random_num1 = rand.random() # first random number
    random_num2 = rand.random() # second random number

    if(random_num1 >= 1/2): # converts the first random number to 0 or 1
        x = 0
    else: x = 1

    if(random_num2 >= 1/2): # converts the second random number to 0 or 1
        y = 0
    else: y = 1
        
    # The main part of Alice and Bob quantum strategy is to fix different rotation angles for their qubit according to the input x,y
    theta = qAlice_output(qA_st, x) # fixes Alice's rotation for her qubit
    phi = qBob_output(qB_st, y) # fixes Bob's rotation for his qubit
    
    # The following gates rotate Alice's qubit and Bob's qubit
    game.ry(theta,0)
    game.ry(phi,1)
    
    # These gates are used to measure  the value of the qubits
    game.measure_all([0,1],[0,1])

    # executes circuit and store the output of the measurements
    result = backend.run(game, shots=shots).result()

    #data = result.get_counts('game') # extract the outcomes and their statistics from the result of the execution
    data = result.get_counts()

    # reads the result of the measurements of the quantum system
    for outcomes in data.keys():
        out = outcomes


    # converts the result of the measurements contained in the classical register as string '00', '01', '10', '11',
    # which are the answers of Alice(a) and Bob (b), from a 'string' type  to 'integer' type 
    if(out == '00'):
        a = 0
        b = 0
    if(out == '01'):
        a = 1
        b = 0    
    if(out == '10'):
        a = 0
        b = 1
    if(out == '11'):
        a = 1
        b = 1

    # check if the condition for winning the game is met
    if(x*y == a^b):
        cont_win += 1 # increase thes won games' counter if the condition to win the game is met
    
    cont_tot += 1 # increases the played games' counter

qProb_win = cont_win/cont_tot

print('Alice and Bob won the game with probability: ', qProb_win*100, '%')

Alice and Bob won the game with probability:  86.5 %


In [8]:
import qiskit.tools.jupyter
%qiskit_version_table